In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler, StandardScaler

sys.path.append(os.path.abspath('../src/'))
from ModelSelection import *
from ActivationFunctions import *
from NeuralNetwork import *
from MyUtils import *

In [2]:
cup_df = pd.read_csv(
    '../data/ML-CUP23-TR.csv',
    index_col=0,
    comment='#',
    header=None,
    names=['feature_' + str(i+1) for i in range(10)] + ['target_' + str(i+1) for i in range(3)]
)

data_set = cup_df.values
len_data = len(data_set)
np.random.shuffle(data_set)

training_len = int(0.8*len_data)

training_set = data_set[:int(training_len)] # Training Set for KFold Cross Validation
test_set = data_set[int(training_len):] # Hold Out Test Set

TR_OUTPUT = 3
TR_INPUT = data_set.shape[1] - TR_OUTPUT

scaler_out = StandardScaler()
scaler_in = StandardScaler()
scaler_in.fit(training_set[:,:TR_INPUT])
scaler_out.fit(training_set[:,TR_INPUT:])

tr_normalized = np.empty((training_len, TR_OUTPUT + TR_INPUT))
tr_normalized[:,:TR_INPUT] = scaler_in.transform(training_set[:,:TR_INPUT])
tr_normalized[:,TR_INPUT:] = scaler_out.transform(training_set[:,TR_INPUT:])

test_normalized = np.empty((len_data - training_len, TR_OUTPUT + TR_INPUT))
test_normalized[:,:TR_INPUT] = scaler_in.transform(test_set[:,:TR_INPUT])
test_normalized[:,TR_INPUT:] = scaler_out.transform(test_set[:,TR_INPUT:])

In [3]:
hidden_len = 32

hidden_fun = 'tanh'
output_fun = 'identity'

tanh_l1 = create_stratified_topology([TR_INPUT,30,TR_OUTPUT], 
                                      [[None,[]]]*TR_INPUT + [[hidden_fun, [1]]]*hidden_len + [[output_fun, []]]*TR_OUTPUT)

hidden_fun = 'sigmoid'
output_fun = 'identity'

sigmoid_l1 = create_stratified_topology([TR_INPUT,30,TR_OUTPUT], 
                                      [[None,[]]]*TR_INPUT + [[hidden_fun, [1]]]*hidden_len + [[output_fun, []]]*TR_OUTPUT)


In [4]:
hidden_fun = 'tanh'
output_fun = 'identity'

tanh_l2 = create_stratified_topology([TR_INPUT,16, 16,TR_OUTPUT], 
                                      [[None,[]]]*TR_INPUT + [[hidden_fun, [1]]]*16 + [[hidden_fun, [1]]]*16 + [[output_fun, []]]*TR_OUTPUT)

hidden_fun = 'ReLU'
output_fun = 'identity'
relu_l2 = create_stratified_topology([TR_INPUT,16, 16,TR_OUTPUT], 
                                      [[None,[]]]*TR_INPUT + [[hidden_fun, [1]]]*16 + [[hidden_fun, [1]]]*16 + [[output_fun, []]]*TR_OUTPUT)

In [5]:
MS = ModelSelection('../data/single_layer.csv')

hyperparam_grid = {
    'learning_rate':[0.01, 0.1, 0.3, 0.5, 0.7],
    'lr_decay_tau':[200],
    'adamax' : [True, False],
    'adamax_learning_rate' : [0.02, 0.01, 0.04, 0.06, 0.1],
    'exp_decay_rate_1':[0.9, 0.8],
    'exp_decay_rate_2':[0.999, 0.5],
    'alpha_momentum':[0, 0.5, 0.9],
    'lambda_tikhonov':[0.00001, 0.0001],
    'batch_size':[int(training_len/15)],
    'min_epochs': [100],
    'max_epochs':[800],
    'metrics':[[ErrorFunctions.mean_euclidean_error, ErrorFunctions.mean_squared_error]],
    'error_increase_tolerance':[0.00001, 0.0001],
    'patience':[10],
    'topology': [str(tanh_l2)],
}

c1 = lambda x: x
constr = {'adamax': (c1, ['learning_rate', 'alpha_momentum', 'lr_decay_tau'], ['adamax_learning_rate', 'exp_decay_rate_1', 'exp_decay_rate_2'])}

MS.grid_searchKF(tr_normalized, hyperparam_grid, constr, 3, 6, True)